In [1]:
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import scipy.io as io
from scipy.io import loadmat
from scipy.io import savemat
import torch.utils.data as data_utils
import torch.optim as optim
import matplotlib.pyplot as plt
import math

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)

C:\Anaconda3\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


cuda:0


In [2]:
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from PIL import Image

try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC

def load_data(path,file):
    name=path+file
    m=loadmat(name)
    x=torch.tensor(m['feature']).to(device)
    return x.float()

def _convert_image_to_rgb(image):
    return image.convert("RGB")

In [6]:
import clip

def _transform(n_px):
    return Compose([
        Resize(n_px, interpolation=BICUBIC),
        CenterCrop(n_px),
        _convert_image_to_rgb,
        ToTensor(),
        Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
    ])

def features(net, x):
    x = x.type(net.conv1.weight.dtype)
    for conv, bn in [(net.conv1, net.bn1), (net.conv2, net.bn2), (net.conv3, net.bn3)]:
        x = F.relu(bn(conv(x)))
    x = net.avgpool(x)
    x = net.layer1(x)
    x = net.layer2(x)
    x = net.layer3(x)
    x = net.layer4(x)
    x = net.attnpool(x)
    return x

print(clip.available_models())
model, preprocess = clip.load('RN50x4', device)

['RN50', 'RN101', 'RN50x4', 'RN50x16', 'RN50x64', 'ViT-B/32', 'ViT-B/16', 'ViT-L/14', 'ViT-L/14@336px']


In [5]:
#Billion-scale semi-supervised learning for image classification, https://arxiv.org/abs/1905.00546
def features(net, x):
    # See note [TorchScript super()]
    x = net.conv1(x)
    x = net.bn1(x)
    x = net.relu(x)
    x = net.maxpool(x)

    x = net.layer1(x)
    x = net.layer2(x)
    x = net.layer3(x)
    x = net.layer4(x)
    
    #print(x.shape)
    x = F.avg_pool2d(x,x.shape[2],1)
    x = torch.flatten(x, 1)
    return x

def _transform(n_px):
    return Compose([
        Resize(n_px, interpolation=BICUBIC),
        CenterCrop(n_px),
        _convert_image_to_rgb,
        ToTensor(),
        Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])

torch.hub.list('facebookresearch/semi-supervised-ImageNet1K-models')
model = torch.hub.load('facebookresearch/semi-supervised-ImageNet1K-models', 'resnet50_swsl')
model=model.to(device)
model.eval()

Using cache found in C:\Users\abarbu/.cache\torch\hub\facebookresearch_semi-supervised-ImageNet1K-models_master
Using cache found in C:\Users\abarbu/.cache\torch\hub\facebookresearch_semi-supervised-ImageNet1K-models_master
C:\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [110]:
from torchvision.datasets import CIFAR100
from torch.utils.data import DataLoader
    
data = CIFAR100(root='c:/datasets/CIFAR100', download=True, train=True,transform=_transform(32))
#data = CIFAR100(root='c:/datasets/CIFAR100', download=True, train=True,transform=preprocess)
loader = DataLoader(data, batch_size=250, shuffle=False, num_workers=0)

Files already downloaded and verified


In [125]:
data = CIFAR100(root='c:/datasets/CIFAR100', download=True, train=False,transform=_transform(256))
loader = DataLoader(data, batch_size=250, shuffle=False, num_workers=0)

Files already downloaded and verified


In [ ]:
# compute features using loader
i=0
for d in loader:
    images,yi=d
    images=images.to(device)
    with torch.no_grad():
        #print(images.shape)
        fi=features32(model.visual,images)    
    if i==0:
        X=fi.cpu()
        y=yi
    else:
        X=torch.cat((X,fi.cpu()),dim=0)
        y=torch.cat((y,yi),dim=0)
    i+=1
    if i%10==0:
        print(i,X.shape)
        
X=X.squeeze()
print(X.shape)

In [3]:
name = 'train.txt'
#name = 'c:/datasets/Imagenet/train.txt'
with open(name, 'r') as f:
    names = f.read().splitlines()
nc=len(names)
print(nc)
classes=torch.load('classes_val.pth')

1000


In [85]:
# compute features for Imagenet and save them
from os import listdir
from os.path import isfile, join
from PIL import Image
from torch.utils.data import Dataset,TensorDataset, DataLoader

def ComputeFeatures(path,files,nx): 
    n=len(files)
    x=torch.zeros([n,3,nx,nx], dtype=torch.float32)
    for i in range(n):
        im = Image.open(join(path, files[i]))
        x[i,:,:,:]=_transform(nx)(im)
    data=TensorDataset(x)
    loader=DataLoader(data,batch_size=256,shuffle=False)
    i=0
    for images in loader:
        images=images[0].to(device)
        with torch.no_grad():
            fi=features32(model.visual,images)    
        if i==0:
            X=fi.cpu()
        else:
            X=torch.cat((X,fi.cpu()),dim=0)
        i+=1
    return X.squeeze()

nx=288
for i in range(nc):
    path='C:/Datasets/ILSVRC2016/ILSVRC/Data/CLS-LOC/train/'+names[i]
    files = [f for f in listdir(path) if isfile(join(path, f))]
    x=ComputeFeatures(path,files,nx)
    name='C:/Datasets/ILSVRC2016/features_2560/'+names[i]+'.mat'
    print(i,name)
    savemat(name,{'feature':x.float().numpy()})

0 C:/Datasets/ILSVRC2016/features_2560/n01440764.mat
1 C:/Datasets/ILSVRC2016/features_2560/n01443537.mat
2 C:/Datasets/ILSVRC2016/features_2560/n01484850.mat
3 C:/Datasets/ILSVRC2016/features_2560/n01491361.mat
4 C:/Datasets/ILSVRC2016/features_2560/n01494475.mat
5 C:/Datasets/ILSVRC2016/features_2560/n01496331.mat
6 C:/Datasets/ILSVRC2016/features_2560/n01498041.mat
7 C:/Datasets/ILSVRC2016/features_2560/n01514668.mat
8 C:/Datasets/ILSVRC2016/features_2560/n01514859.mat
9 C:/Datasets/ILSVRC2016/features_2560/n01518878.mat
10 C:/Datasets/ILSVRC2016/features_2560/n01530575.mat
11 C:/Datasets/ILSVRC2016/features_2560/n01531178.mat
12 C:/Datasets/ILSVRC2016/features_2560/n01532829.mat
13 C:/Datasets/ILSVRC2016/features_2560/n01534433.mat
14 C:/Datasets/ILSVRC2016/features_2560/n01537544.mat
15 C:/Datasets/ILSVRC2016/features_2560/n01558993.mat
16 C:/Datasets/ILSVRC2016/features_2560/n01560419.mat
17 C:/Datasets/ILSVRC2016/features_2560/n01580077.mat
18 C:/Datasets/ILSVRC2016/features_256

151 C:/Datasets/ILSVRC2016/features_2560/n02085620.mat
152 C:/Datasets/ILSVRC2016/features_2560/n02085782.mat
153 C:/Datasets/ILSVRC2016/features_2560/n02085936.mat
154 C:/Datasets/ILSVRC2016/features_2560/n02086079.mat
155 C:/Datasets/ILSVRC2016/features_2560/n02086240.mat
156 C:/Datasets/ILSVRC2016/features_2560/n02086646.mat
157 C:/Datasets/ILSVRC2016/features_2560/n02086910.mat
158 C:/Datasets/ILSVRC2016/features_2560/n02087046.mat
159 C:/Datasets/ILSVRC2016/features_2560/n02087394.mat
160 C:/Datasets/ILSVRC2016/features_2560/n02088094.mat
161 C:/Datasets/ILSVRC2016/features_2560/n02088238.mat
162 C:/Datasets/ILSVRC2016/features_2560/n02088364.mat
163 C:/Datasets/ILSVRC2016/features_2560/n02088466.mat
164 C:/Datasets/ILSVRC2016/features_2560/n02088632.mat
165 C:/Datasets/ILSVRC2016/features_2560/n02089078.mat
166 C:/Datasets/ILSVRC2016/features_2560/n02089867.mat
167 C:/Datasets/ILSVRC2016/features_2560/n02089973.mat
168 C:/Datasets/ILSVRC2016/features_2560/n02090379.mat
169 C:/Dat

300 C:/Datasets/ILSVRC2016/features_2560/n02165105.mat
301 C:/Datasets/ILSVRC2016/features_2560/n02165456.mat
302 C:/Datasets/ILSVRC2016/features_2560/n02167151.mat
303 C:/Datasets/ILSVRC2016/features_2560/n02168699.mat
304 C:/Datasets/ILSVRC2016/features_2560/n02169497.mat
305 C:/Datasets/ILSVRC2016/features_2560/n02172182.mat
306 C:/Datasets/ILSVRC2016/features_2560/n02174001.mat
307 C:/Datasets/ILSVRC2016/features_2560/n02177972.mat
308 C:/Datasets/ILSVRC2016/features_2560/n02190166.mat
309 C:/Datasets/ILSVRC2016/features_2560/n02206856.mat
310 C:/Datasets/ILSVRC2016/features_2560/n02219486.mat
311 C:/Datasets/ILSVRC2016/features_2560/n02226429.mat
312 C:/Datasets/ILSVRC2016/features_2560/n02229544.mat
313 C:/Datasets/ILSVRC2016/features_2560/n02231487.mat
314 C:/Datasets/ILSVRC2016/features_2560/n02233338.mat
315 C:/Datasets/ILSVRC2016/features_2560/n02236044.mat
316 C:/Datasets/ILSVRC2016/features_2560/n02256656.mat
317 C:/Datasets/ILSVRC2016/features_2560/n02259212.mat
318 C:/Dat

449 C:/Datasets/ILSVRC2016/features_2560/n02859443.mat
450 C:/Datasets/ILSVRC2016/features_2560/n02860847.mat
451 C:/Datasets/ILSVRC2016/features_2560/n02865351.mat
452 C:/Datasets/ILSVRC2016/features_2560/n02869837.mat
453 C:/Datasets/ILSVRC2016/features_2560/n02870880.mat
454 C:/Datasets/ILSVRC2016/features_2560/n02871525.mat
455 C:/Datasets/ILSVRC2016/features_2560/n02877765.mat
456 C:/Datasets/ILSVRC2016/features_2560/n02879718.mat
457 C:/Datasets/ILSVRC2016/features_2560/n02883205.mat
458 C:/Datasets/ILSVRC2016/features_2560/n02892201.mat
459 C:/Datasets/ILSVRC2016/features_2560/n02892767.mat
460 C:/Datasets/ILSVRC2016/features_2560/n02894605.mat
461 C:/Datasets/ILSVRC2016/features_2560/n02895154.mat
462 C:/Datasets/ILSVRC2016/features_2560/n02906734.mat
463 C:/Datasets/ILSVRC2016/features_2560/n02909870.mat
464 C:/Datasets/ILSVRC2016/features_2560/n02910353.mat
465 C:/Datasets/ILSVRC2016/features_2560/n02916936.mat
466 C:/Datasets/ILSVRC2016/features_2560/n02917067.mat
467 C:/Dat

598 C:/Datasets/ILSVRC2016/features_2560/n03529860.mat
599 C:/Datasets/ILSVRC2016/features_2560/n03530642.mat
600 C:/Datasets/ILSVRC2016/features_2560/n03532672.mat
601 C:/Datasets/ILSVRC2016/features_2560/n03534580.mat
602 C:/Datasets/ILSVRC2016/features_2560/n03535780.mat
603 C:/Datasets/ILSVRC2016/features_2560/n03538406.mat
604 C:/Datasets/ILSVRC2016/features_2560/n03544143.mat
605 C:/Datasets/ILSVRC2016/features_2560/n03584254.mat
606 C:/Datasets/ILSVRC2016/features_2560/n03584829.mat
607 C:/Datasets/ILSVRC2016/features_2560/n03590841.mat
608 C:/Datasets/ILSVRC2016/features_2560/n03594734.mat
609 C:/Datasets/ILSVRC2016/features_2560/n03594945.mat
610 C:/Datasets/ILSVRC2016/features_2560/n03595614.mat
611 C:/Datasets/ILSVRC2016/features_2560/n03598930.mat
612 C:/Datasets/ILSVRC2016/features_2560/n03599486.mat
613 C:/Datasets/ILSVRC2016/features_2560/n03602883.mat
614 C:/Datasets/ILSVRC2016/features_2560/n03617480.mat
615 C:/Datasets/ILSVRC2016/features_2560/n03623198.mat
616 C:/Dat

747 C:/Datasets/ILSVRC2016/features_2560/n04023962.mat
748 C:/Datasets/ILSVRC2016/features_2560/n04026417.mat
749 C:/Datasets/ILSVRC2016/features_2560/n04033901.mat
750 C:/Datasets/ILSVRC2016/features_2560/n04033995.mat
751 C:/Datasets/ILSVRC2016/features_2560/n04037443.mat
752 C:/Datasets/ILSVRC2016/features_2560/n04039381.mat
753 C:/Datasets/ILSVRC2016/features_2560/n04040759.mat
754 C:/Datasets/ILSVRC2016/features_2560/n04041544.mat
755 C:/Datasets/ILSVRC2016/features_2560/n04044716.mat
756 C:/Datasets/ILSVRC2016/features_2560/n04049303.mat
757 C:/Datasets/ILSVRC2016/features_2560/n04065272.mat
758 C:/Datasets/ILSVRC2016/features_2560/n04067472.mat
759 C:/Datasets/ILSVRC2016/features_2560/n04069434.mat
760 C:/Datasets/ILSVRC2016/features_2560/n04070727.mat
761 C:/Datasets/ILSVRC2016/features_2560/n04074963.mat
762 C:/Datasets/ILSVRC2016/features_2560/n04081281.mat
763 C:/Datasets/ILSVRC2016/features_2560/n04086273.mat
764 C:/Datasets/ILSVRC2016/features_2560/n04090263.mat
765 C:/Dat

896 C:/Datasets/ILSVRC2016/features_2560/n04553703.mat
897 C:/Datasets/ILSVRC2016/features_2560/n04554684.mat
898 C:/Datasets/ILSVRC2016/features_2560/n04557648.mat
899 C:/Datasets/ILSVRC2016/features_2560/n04560804.mat
900 C:/Datasets/ILSVRC2016/features_2560/n04562935.mat
901 C:/Datasets/ILSVRC2016/features_2560/n04579145.mat
902 C:/Datasets/ILSVRC2016/features_2560/n04579432.mat
903 C:/Datasets/ILSVRC2016/features_2560/n04584207.mat
904 C:/Datasets/ILSVRC2016/features_2560/n04589890.mat
905 C:/Datasets/ILSVRC2016/features_2560/n04590129.mat
906 C:/Datasets/ILSVRC2016/features_2560/n04591157.mat
907 C:/Datasets/ILSVRC2016/features_2560/n04591713.mat
908 C:/Datasets/ILSVRC2016/features_2560/n04592741.mat
909 C:/Datasets/ILSVRC2016/features_2560/n04596742.mat
910 C:/Datasets/ILSVRC2016/features_2560/n04597913.mat
911 C:/Datasets/ILSVRC2016/features_2560/n04599235.mat
912 C:/Datasets/ILSVRC2016/features_2560/n04604644.mat
913 C:/Datasets/ILSVRC2016/features_2560/n04606251.mat
914 C:/Dat

In [8]:
from os import listdir
from os.path import isfile, join
import xml.etree.ElementTree as ET
def ParseAnoXml(path):
    files = [f for f in listdir(path) if isfile(join(path, f))]
    classes={}
    nf=len(files)
    for i in range(nf):
        f=files[i]
        tree = ET.parse(path+f)
        root = tree.getroot()
        child=root.findall("./object/name")[0]
        fj=f[0:-4]+'.jpeg'
        if child.text in classes.keys():
            classes[child.text].append(fj)
        else:
            classes[child.text]=[fj]
        if i%1000==0:
            print(i,fj)
    return classes
path='d:/Datasets/ILSVRC2016/ILSVRC/Annotations/CLS-LOC/val/'
classes=ParseAnoXml(path)
torch.save(classes,'classes_val.pth')
print('done')

0 ILSVRC2012_val_00000001.jpeg
1000 ILSVRC2012_val_00001001.jpeg
2000 ILSVRC2012_val_00002001.jpeg
3000 ILSVRC2012_val_00003001.jpeg
4000 ILSVRC2012_val_00004001.jpeg
5000 ILSVRC2012_val_00005001.jpeg
6000 ILSVRC2012_val_00006001.jpeg
7000 ILSVRC2012_val_00007001.jpeg
8000 ILSVRC2012_val_00008001.jpeg
9000 ILSVRC2012_val_00009001.jpeg
10000 ILSVRC2012_val_00010001.jpeg
11000 ILSVRC2012_val_00011001.jpeg
12000 ILSVRC2012_val_00012001.jpeg
13000 ILSVRC2012_val_00013001.jpeg
14000 ILSVRC2012_val_00014001.jpeg
15000 ILSVRC2012_val_00015001.jpeg
16000 ILSVRC2012_val_00016001.jpeg
17000 ILSVRC2012_val_00017001.jpeg
18000 ILSVRC2012_val_00018001.jpeg
19000 ILSVRC2012_val_00019001.jpeg
20000 ILSVRC2012_val_00020001.jpeg
21000 ILSVRC2012_val_00021001.jpeg
22000 ILSVRC2012_val_00022001.jpeg
23000 ILSVRC2012_val_00023001.jpeg
24000 ILSVRC2012_val_00024001.jpeg
25000 ILSVRC2012_val_00025001.jpeg
26000 ILSVRC2012_val_00026001.jpeg
27000 ILSVRC2012_val_00027001.jpeg
28000 ILSVRC2012_val_00028001.jpe

In [40]:
from os import listdir
from os.path import isfile, join
from PIL import Image
from torch.utils.data import Dataset,TensorDataset, DataLoader
def ComputeFeatures(path,files,nx): 
    n=len(files)
    x=torch.zeros([n,3,nx,nx], dtype=torch.float32)
    for i in range(n):
        im = Image.open(join(path, files[i]))
        x[i,:,:,:]=_transform(nx)(im)
    data=TensorDataset(x)
    loader=DataLoader(data,batch_size=256,shuffle=False)
    i=0
    for images in loader:
        images=images[0].to(device)
        with torch.no_grad():
            fi=features(model.visual,images)    
        if i==0:
            X=fi
        else:
            X=torch.cat((X,fi),dim=0)
        i+=1
    return X.squeeze().float()
# save test features
nx=288
print(nx)
for i in range(nc):
    path='C:/Datasets/ILSVRC2016/ILSVRC/Data/CLS-LOC/val/'
    files=classes[names[i]]
    x=ComputeFeatures(path,files,nx)
    name='C:/Datasets/ILSVRC2016/features_val_640/'+names[i]+'.mat'
    if i%100==0:
        print(i)
    savemat(name,{'feature':x.cpu().float().numpy()})

288
0
100
200
300
400
500
600
700
800
900


In [ ]:
# compute features and train PCAs
# save model for each class separately
from os import listdir
from os.path import isfile, join
from PIL import Image
from torch.utils.data import Dataset,TensorDataset, DataLoader

def ComputeFeatures(path,files,nx): 
    n=len(files)
    x=torch.zeros([n,3,nx,nx], dtype=torch.float32)
    for i in range(n):
        im = Image.open(join(path, files[i]))
        x[i,:,:,:]=_transform(nx)(im)
    data=TensorDataset(x)
    loader=DataLoader(data,batch_size=256,shuffle=False)
    i=0
    for images in loader:
        images=images[0].to(device)
        with torch.no_grad():
            fi=features(model.visual,images)    
        if i==0:
            X=fi
        else:
            X=torch.cat((X,fi),dim=0)
        i+=1
    return X.squeeze().float()

nx=288
nc=1000
k=400  # number of Pcs
N=0.1
la=N**2
for i in range(nc):
    #x=load_data(names[i])
    path='C:/Datasets/ILSVRC2016/ILSVRC/Data/CLS-LOC/train/'+names[i]
    files = [f for f in listdir(path) if isfile(join(path, f))]
    x=ComputeFeatures(path,files,nx)
    mx=torch.mean(x,dim=0)
    u,s,v = torch.linalg.svd(x-mx)
    L=v[0:k,:]
    S=s[0:k]
    name='C:/Datasets/ILSVRC2016/Model/'+names[i]+'.pth'
    torch.save({'mx':mx,'L':L, 'S':S},name)    
    if i%50==0:
        print(i)
print('done')

In [18]:
# train PCAs
nc=len(names)
k=100  # number of Pcs
N=0.1
la=N**2
path='c:/datasets/ImageNet/features_640/'
n=0
for i in range(nc):
    x=load_data(path,names[i])
    n+=x.shape[0]
    if (i%1000==999):
        print(n)
    continue
    #x=X[y==i,:].to(device)
    if i==0:
        d=x.shape[1]
        e=torch.eye(d,device=device)
        mx=torch.zeros((nc,d),device=device)
        L=torch.zeros((nc,k,d),device=device)
        S=torch.zeros((nc,k),device=device)
    mx[i,:]=torch.mean(x,dim=0)
    u,s,v = torch.linalg.svd(x-mx[i,:])
    L[i,:,:]=v[0:k,:]
    S[i,:]=s[0:k]
    if i%100==0:
        print(i)
print('done')
print(n)
#torch.save({'mx':mx,'L':L, 'S':S},'c:/tmp/pca_Imagenet.pth')

1082869
2204762
3294682
4383413
5468356
6543512
7639423
8639651
9677249
10634326
done
11060192


In [ ]:
# train PCAs
# save model separately for each class
nc=len(names)
k=400  # number of Pcs
N=0.1
la=N**2
path='c:/datasets/ILSVRC2016/features_640/'
for i in range(nc):
    x=load_data(path,names[i])
    #x=X[y==i,:].to(device)
    mx=torch.mean(x,dim=0)
    u,s,v = torch.linalg.svd(x-mx)
    L=v[0:k,:]
    S=s[0:k]
    name='C:/Datasets/Imagenet/Model/'+names[i]+'.pth'
    torch.save({'mx':mx,'L':L, 'S':S},name)    
    if i%500==0:
        print(i)
print('done')

In [5]:
# load the models from a single file
m=torch.load('pca640_288_Imagenet.pth')
#m=torch.load('d:/tmp/pca640_Imagenet.pth')
mx=m['mx']
print(m['L'].shape)
k=20
L=m['L'][:,0:k,:]
S=m['S'][:,0:k]
#torch.save({'mx':mx,'L':L, 'S':S},'pca640_288_Imagenet.pth')

torch.Size([1000, 20, 640])


In [18]:
torch.save({'mx':mx,'L':L.cpu().to(device), 'S':S.cpu().to(device)},'pca640_288_Imagenet.pth')

In [20]:
# load the models saved separately
nc=len(names)
k=100  # number of Pcs
for i in range(nc):
    name='C:/Datasets/Imagenet/Model/'+names[i]+'.pth'
    m=torch.load(name)
    Li=m['L']
    if i==0:
        mx=torch.zeros((nc,Li.shape[1]),dtype=torch.float32)
        L=torch.zeros((nc,k,Li.shape[1]),dtype=torch.float32)
        S=torch.zeros((nc,k),dtype=torch.float32)
    mx[i,:]=m['mx'].float().cpu()
    L[i,:,:]=m['L'][0:k,:].float().cpu()
    S[i,:]=m['S'][0:k].cpu()
    if i%1000==0:
        print(i)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000


In [121]:
la=0.01
i=0
py=classify(X,mx.cpu(),L.cpu(),S.cpu(),la)
print('done')
confmat=confusion_matrix(y.cpu().numpy(),py.cpu().numpy())
print(100-np.sum(np.diag(confmat))/np.sum(confmat)*100)
cv2.imwrite('c:/tmp/conf.png',confmat)

done
70.68


True

In [33]:
k=10
L=L[:,0:k,:]
S=S[:,0:k]

In [19]:
si=compute_sigma(L[0,:,:],S[0,:],0.01)
ld=(640-k)*np.log(0.01)+torch.sum(torch.log(S[0,:]**2+0.01))
print(ld,torch.logdet(si))

tensor(-2767.4937, device='cuda:0') tensor(-2767.4941, device='cuda:0')


In [6]:
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import confusion_matrix
#import cv2


def load_datat(i):
    #name='c:/datasets/ILSVRC2016/Features_val_640/%d.mat'%(i+1)
    name='d:/datasets/ILSVRC2016/features_val_640/'+names[i]
    #name='d:/datasets/Imagenet/features_val_640/'+names[i]
    m=loadmat(name)
    x=torch.tensor(m['feature']).float().to(device)
    return x

def response(x1,mu,L,S,la):
    # response using L,S (fast), using the math computation above
    # xSigma^-1 x = x^Tx/lambda - u^Tu/lambda where u = S/(S^2+lambda)^0.5Lx
    x=x1-mu
    Lx=x@L.t()
    Qx=(S/torch.sqrt(S**2+la)).unsqueeze(0)*Lx
    return (torch.sum(x**2,dim=1)-torch.sum(Qx**2,dim=1))/la

def response1(x1,mu):
    # response using L,S (fast), using the math computation above
    # xSigma^-1 x = x^Tx/lambda - u^Tu/lambda where u = S/(S^2+lambda)^0.5Lx
    x=x1-mu
    return torch.sum(x**2,dim=1)

def compute_sigma(L,S,la):
    e=torch.eye(L.shape[1],device=L.device)
    s2L=(S**2).unsqueeze(1)*L #W=SL
    si = L.t()@s2L
    si+= e*la #Sigma=L^TS^2L+lambda I
    return si

def compute_si(x):
    u,s,v = torch.linalg.svd(x)
    L=v[0:k,:]
    S=s[0:k]
    s2L=(S**2).unsqueeze(1)*L #W=SL
    si = L.t()@s2L
    si+= e*N**2 #Sigma=L^TS^2L+lambda I
    return torch.inverse(si)

def response_si(x1,mu,si):
    # response using sigma inverse(slow)
    # r=(x-mu)Sigma^-1(x-mu), where Sigma=L^TS^2L+lambda I
    x=x1-mu
    six = x@si
    return torch.sum(six*x,axis=1)

def compute_si1(x):
    # sigma inverse using the math computation, without actually computing the inverse
    # v Sigma v^T=v(L^TS^2L+lambda I)v^T=diag(S^2,lambda...) so
    # Sigma = v^T diag(S^2,lambda...) v and Sigma^-1 = v^T diag(S^-2,1/lambda...) v and
    # Sigma^-1 = 1/lambda I - L^T (1/lambda-1/(S^2+lambda)) L
    # Sigma^-1 = 1/lambda I - L^T (S^2/lambda(S^2+lambda)) L
    u,s,v = torch.linalg.svd(x)
    la=N**2
    L=v[0:k,:]
    S=s[0:k]
    s2L=(S**2/(S**2+la)/la).unsqueeze(1)*L
    si = -L.t()@s2L
    si+= e*(1/la)
    return si

def classify(x,mx,L,S,la):
    nj=x.shape[0]
    nc=mx.shape[0]
    k=L.shape[1]
    d=L.shape[2]
    out=torch.zeros(nj,nc,device=device)
    for i in range(nc):
        #ld=(d-k)*np.log(la)+torch.sum(torch.log(S[i,:]**2+la))
        r=response(x,mx[i,:].to(device),L[i,:,:].to(device),S[i,:].to(device),la)
#        r=response1(x,mx[i,:].to(device))
        out[:,i] = r#+ld
    py=torch.argmin(out,dim=1)
    return py


la=0.01
i=0
emptyx=1
for j in range(nc):
    x=load_datat(j)
    if emptyx:
        allx=x
        emptyx=0
    else:
        allx=torch.cat((allx,x),dim=0)
    l=j#label[names[j][0:9]]
    if j==0:
        y=torch.zeros(x.shape[0])+l
    else:
        y=torch.cat((y,torch.zeros(x.shape[0])+l))
    if (j%1000==999)|(j==nc-1):
        print(j+1)    
        pyj=classify(allx,mx,L,S,la)
        emptyx=1
        if i==0:
            py=pyj 
            i=1
        else:
            py=torch.cat((py,pyj),dim=0)
print('done')
confmat=confusion_matrix(y.cpu().numpy(),py.cpu().numpy())
print('accuracy=',np.sum(np.diag(confmat))/np.sum(confmat)*100)
#r, c = linear_sum_assignment(-confmat)
#print('accuracy=',np.sum(confmat[r,c])/np.sum(confmat)*100)
#cv2.imwrite('c:/tmp/conf.png',confmat)

1000
done
accuracy= 71.25
